Code to warm up kernel

In [1]:
#Imports
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pytrends.request import TrendReq
from arch import arch_model
#Declare constants
ETH_TICKER = "ETH-USD"
SPX_TICKER = "^GSPC" 
BTC_TICKER = "BTC-USD"

#Timeframe for data collection
START_DATE = "2022-01-01"
END_DATE = "2025-12-07"


Below is the code to get data for:

Bitcoin & Ether

In [2]:

def clean_index(df):
    """Safely normalizes index and removes timezone if it exists."""
    df.index = df.index.normalize()
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)
    return df

# --- FETCH ETH DATA ---
# ETH trades 24/7, so this will be your primary (left) dataset
eth_data = yf.Ticker(ETH_TICKER).history(start=START_DATE, end=END_DATE)
df_eth = eth_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
df_combined_daily_eth = clean_index(df_eth)
        
# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_eth.info()


--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1436 entries, 2022-01-01 to 2025-12-06
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1436 non-null   float64
 1   High    1436 non-null   float64
 2   Low     1436 non-null   float64
 3   Close   1436 non-null   float64
 4   Volume  1436 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 67.3 KB


In [3]:

# --- FETCH BTC DATA ---
# BTC trades 24/7, so this will be your primary (left) dataset
btc_data = yf.Ticker(BTC_TICKER).history(start=START_DATE, end = END_DATE)
df_btc = btc_data[['Open', 'High', 'Low', 'Close','Volume']].copy()
df_combined_daily_btc = clean_index(df_btc)  

df_combined_daily_btc = df_btc

# Display final data types and non-null counts
print("\n--- Final DataFrame Info ---")
df_combined_daily_btc.info()


--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1436 entries, 2022-01-01 to 2025-12-06
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1436 non-null   float64
 1   High    1436 non-null   float64
 2   Low     1436 non-null   float64
 3   Close   1436 non-null   float64
 4   Volume  1436 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 67.3 KB


BTC HASHRATE + Fear & Greed index ~

All using dataset

In [4]:
FILE_PATH = r"C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\hashrateBTC.csv"
START_DATE = '2022-01-01 00:00:00'
END_DATE = '2025-12-07 00:00:00'

df_hashrate_data = pd.read_csv(
    FILE_PATH,
    sep=';',
    names = ['DateTime', 'Hash Rate'],
    index_col='DateTime',
    parse_dates=['DateTime'],
    skiprows=1, 
    decimal=',',
    header=None
    )
df_hashrate = df_hashrate_data.loc[START_DATE:END_DATE].copy()


df_hashrate.index = pd.to_datetime(df_hashrate.index)

df_hashrate['Hash Rate'] = pd.to_numeric(df_hashrate['Hash Rate'].astype(float))
daily_avg_hash = df_hashrate.resample('D').mean()
print("successfully extracted hashrate data")

daily_avg_hash.head()


successfully extracted hashrate data


,Hash Rate
DateTime,
2022-01-01,2.075325e+20
2022-01-02,1.906403e+20
2022-01-03,1.689218e+20
2022-01-04,1.773679e+20
2022-01-05,1.809876e+20


In [5]:
FILE_PATH_FG = r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\btc_with_fgi_4h.csv'##
#START_DATE = '2023-01-01 00:00:00'
#END_DATE = '2025-12-07 00:00:00'
df_fng_data = pd.read_csv(
    FILE_PATH_FG,
    sep=',',
    index_col='timestamp',
    )
df_fng_data.index = pd.to_datetime(df_fng_data.index)                                   
df_fng_data.head()


,open,close,high,low,Fear & Greed Index,Fear & Greed Classification
timestamp,,,,,,
2025-12-13 04:00:00,90261.0,90362.0,90436.0,90136.0,23.0,Extreme Fear
2025-12-13 00:00:00,90195.0,90307.0,90397.0,89966.0,23.0,Extreme Fear
2025-12-12 20:00:00,89947.0,90232.0,90615.0,89600.0,29.0,Fear
2025-12-12 16:00:00,92405.0,90160.0,92511.0,90160.0,29.0,Fear
2025-12-12 12:00:00,92418.0,92387.0,92587.0,92126.0,29.0,Fear


In [6]:
df_fng_data.drop(['open','close','high','low','Fear & Greed Classification'], axis=1, inplace=True)
df_fng_data.fillna(method='ffill', inplace=True)

C:\Users\Kevin Ivander\AppData\Local\Temp\ipykernel_18092\786426176.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_fng_data.fillna(method='ffill', inplace=True)


Google Trends data!

In [7]:
df_gtrends_daily = pd.read_csv(r'C:\Users\Kevin Ivander\Documents\GitHub\DSC101\csv_files\google_trends_daily_20251213.csv')
df_gtrends_daily['date'] = pd.to_datetime(df_gtrends_daily['date'])
df_gtrends_daily.set_index('date', inplace=True)

df_gtrends_daily.tail()

,GTrends_BTC,GTrends_ETH
date,,
2025-12-03,45,6
2025-12-04,45,6
2025-12-05,45,6
2025-12-06,45,6
2025-12-07,38,5


In [8]:
df_gtrends_eth = df_gtrends_daily['GTrends_ETH'].copy()
df_gtrends_eth.head()

date
2022-01-01    12
2022-01-02    16
2022-01-03    16
2022-01-04    16
2022-01-05    16
Name: GTrends_ETH, dtype: int64

In [9]:
df_gtrends_btc = df_gtrends_daily['GTrends_BTC'].copy()

df_gtrends_btc.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1437 entries, 2022-01-01 to 2025-12-07
Series name: GTrends_BTC
Non-Null Count  Dtype
--------------  -----
1437 non-null   int64
dtypes: int64(1)
memory usage: 22.5 KB


Sychronizing and Scaling

In [10]:
#BTC
df_combined_daily_btc = df_combined_daily_btc.join(df_fng_data)
df_combined_daily_btc = df_combined_daily_btc.join(daily_avg_hash)
df_combined_daily_btc = df_combined_daily_btc.join(df_gtrends_btc)
#ETH
df_combined_daily_eth = df_combined_daily_eth.join(df_fng_data)
df_combined_daily_eth = df_combined_daily_eth.join(df_gtrends_eth)

In [11]:
df_combined_daily_btc['GTrends_BTC'] = (
    df_combined_daily_btc['GTrends_BTC'].ffill()
)


In [12]:
df_combined_daily_eth['GTrends_ETH'] = (
    df_combined_daily_eth['GTrends_ETH'].ffill()
)

In [13]:

# Convert raw HashRate to Exahashes (EH/s) by dividing by 10^18 and volume into billions
df_combined_daily_btc['Volume'] = df_combined_daily_btc['Volume'] / 1e10
df_combined_daily_eth['Volume'] = df_combined_daily_eth['Volume'] / 1e10
df_combined_daily_btc['Hashrate'] = df_combined_daily_btc['Hash Rate'] / 1e18

# Rename the column so you don't forget the unit later
df_combined_daily_btc.rename(columns={'Hashrate': 'Hashrate_EH'}, inplace=True)
df_combined_daily_btc.rename(columns={'Volume': 'Volume (billions)'}, inplace=True)
df_combined_daily_eth.rename(columns={'Volume': 'Volume (billions)'}, inplace=True)
print(df_combined_daily_btc.head().to_markdown())
print(df_combined_daily_eth.head().to_markdown())


| Date                |    Open |    High |     Low |   Close |   Volume (billions) |   Fear & Greed Index |   Hash Rate |   GTrends_BTC |   Hashrate_EH |
|:--------------------|--------:|--------:|--------:|--------:|--------------------:|---------------------:|------------:|--------------:|--------------:|
| 2022-01-01 00:00:00 | 46311.7 | 47827.3 | 46288.5 | 47686.8 |             2.45827 |                   21 | 2.07532e+20 |            47 |       207.532 |
| 2022-01-02 00:00:00 | 47680.9 | 47881.4 | 46856.9 | 47345.2 |             2.79516 |                   29 | 1.9064e+20  |            61 |       190.64  |
| 2022-01-03 00:00:00 | 47343.5 | 47510.7 | 45836   | 46458.1 |             3.30716 |                   29 | 1.68922e+20 |            61 |       168.922 |
| 2022-01-04 00:00:00 | 46458.9 | 47406.5 | 45752.5 | 45897.6 |             4.24947 |                   23 | 1.77368e+20 |            61 |       177.368 |
| 2022-01-05 00:00:00 | 45899.4 | 46929   | 42798.2 | 43569   |       

In [14]:
print("For ETH:")
df_combined_daily_eth.info()
print("For BTC:")
df_combined_daily_btc.info()

For ETH:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1436 entries, 2022-01-01 to 2025-12-06
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                1436 non-null   float64
 1   High                1436 non-null   float64
 2   Low                 1436 non-null   float64
 3   Close               1436 non-null   float64
 4   Volume (billions)   1436 non-null   float64
 5   Fear & Greed Index  1436 non-null   float64
 6   GTrends_ETH         1436 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 89.8 KB
For BTC:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1436 entries, 2022-01-01 to 2025-12-06
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                1436 non-null   float64
 1   High                1436 non-null   float64
 2   Low                 1436 non-null   float

In [15]:
df_combined_daily_eth['Range'] = (
    df_combined_daily_eth['High'] - df_combined_daily_eth['Low']
)

df_combined_daily_eth['Return'] = (
    df_combined_daily_eth['Close'].pct_change()
)

df_combined_daily_btc['Range'] = (
    df_combined_daily_btc['High'] - df_combined_daily_btc['Low']
)

df_combined_daily_btc['Return'] = (
    df_combined_daily_btc['Close'].pct_change()
)


In [16]:
df_combined_daily_btc.describe()

,Open,High,Low,Close,Volume (billions),Fear & Greed Index,Hash Rate,GTrends_BTC,Hashrate_EH,Range,Return
count,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1.436000e+03,1436.000000,1436.000000,1436.000000,1435.000000
mean,55577.861932,56556.397494,54557.713611,55608.165663,3.453537,48.494429,5.319140e+20,40.263231,531.914032,1998.683882,0.000805
std,32323.365664,32806.045244,31795.235216,32334.693054,2.195903,20.520118,2.762110e+20,14.395085,276.210974,1720.802053,0.027143
min,15782.300781,16253.047852,15599.046875,15787.284180,0.533117,6.000000,1.564582e+20,23.000000,156.458195,60.826172,-0.159747
25%,27200.549805,27588.646973,26765.235352,27206.511230,1.903023,29.000000,2.709155e+20,30.000000,270.915482,752.050781,-0.011987
50%,43384.212891,44087.222656,42657.287109,43367.759766,2.896271,50.000000,4.971694e+20,35.000000,497.169450,1527.923828,-0.000252
75%,84357.519531,85802.796875,83111.703125,84392.457031,4.314902,67.000000,7.387168e+20,48.000000,738.716792,2679.125000,0.013117
max,124752.140625,126198.070312,123196.046875,124752.531250,18.174642,94.000000,1.305500e+21,100.000000,1305.499723,17927.250000,0.145412


In [17]:
print(df_combined_daily_btc.head().to_markdown())
print(df_combined_daily_eth.head().to_markdown())

| Date                |    Open |    High |     Low |   Close |   Volume (billions) |   Fear & Greed Index |   Hash Rate |   GTrends_BTC |   Hashrate_EH |   Range |       Return |
|:--------------------|--------:|--------:|--------:|--------:|--------------------:|---------------------:|------------:|--------------:|--------------:|--------:|-------------:|
| 2022-01-01 00:00:00 | 46311.7 | 47827.3 | 46288.5 | 47686.8 |             2.45827 |                   21 | 2.07532e+20 |            47 |       207.532 | 1538.83 | nan          |
| 2022-01-02 00:00:00 | 47680.9 | 47881.4 | 46856.9 | 47345.2 |             2.79516 |                   29 | 1.9064e+20  |            61 |       190.64  | 1024.47 |  -0.00716327 |
| 2022-01-03 00:00:00 | 47343.5 | 47510.7 | 45836   | 46458.1 |             3.30716 |                   29 | 1.68922e+20 |            61 |       168.922 | 1674.76 |  -0.0187369  |
| 2022-01-04 00:00:00 | 46458.9 | 47406.5 | 45752.5 | 45897.6 |             4.24947 |               

In [18]:

df_combined_daily_eth = df_combined_daily_eth.dropna()
df_combined_daily_btc = df_combined_daily_btc.dropna()
print("ETH shape:", df_combined_daily_eth.shape)
print("BTC shape:", df_combined_daily_btc.shape)

df_combined_daily_eth.head()

ETH shape: (1435, 9)
BTC shape: (1435, 11)


,Open,High,Low,Close,Volume (billions),Fear & Greed Index,GTrends_ETH,Range,Return
Date,,,,,,,,,
2022-01-02,3769.298340,3836.161865,3727.357422,3829.564941,0.988147,29.0,16,108.804443,0.015881
2022-01-03,3829.535645,3836.198730,3698.047607,3761.380371,1.208078,29.0,16,138.151123,-0.017805
2022-01-04,3761.361572,3876.785156,3723.349854,3794.056641,1.403093,23.0,16,153.435303,0.008687
2022-01-05,3794.269043,3842.059814,3456.745361,3550.386963,1.806134,24.0,16,385.314453,-0.064224
2022-01-06,3549.708984,3549.708984,3335.376709,3418.408203,2.429379,15.0,16,214.332275,-0.037173


In [19]:
df_eth = df_combined_daily_eth.copy()

df_eth['Direction'] = (
    df_eth['Close'].shift(-1) > df_eth['Close']
).astype(int)

df_eth.dropna(inplace=True)


In [20]:
df_btc = df_combined_daily_btc.copy()

df_btc['Direction'] = (
    df_btc['Close'].shift(-1) > df_btc['Close']
).astype(int)

df_btc.dropna(inplace=True)


In [21]:
print("ETH shape:", df_combined_daily_eth.shape)
print("BTC shape:", df_combined_daily_btc.shape)

ETH shape: (1435, 9)
BTC shape: (1435, 11)


START OF LEARNING MODEL

Predicting Close price of BTC

In [22]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


ETH

In [27]:
X = df_eth[
    ['Open','High','Low',
     'Volume (billions)',
     'Fear & Greed Index',
     'GTrends_ETH',
     'Range','Return']
]

y = df_eth['Direction']

In [29]:
split_idx = int(len(df_eth) * 0.8)

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test  = y.iloc[split_idx:]

In [31]:
X_train.head()

,Open,High,Low,Volume (billions),Fear & Greed Index,GTrends_ETH,Range,Return
Date,,,,,,,,
2022-01-02,3769.298340,3836.161865,3727.357422,0.988147,29.0,16,108.804443,0.015881
2022-01-03,3829.535645,3836.198730,3698.047607,1.208078,29.0,16,138.151123,-0.017805
2022-01-04,3761.361572,3876.785156,3723.349854,1.403093,23.0,16,153.435303,0.008687
2022-01-05,3794.269043,3842.059814,3456.745361,1.806134,24.0,16,385.314453,-0.064224
2022-01-06,3549.708984,3549.708984,3335.376709,2.429379,15.0,16,214.332275,-0.037173


In [32]:
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

print("Train date range:", X_train.index.min(), "→", X_train.index.max())
print("Test date range:", X_test.index.min(), "→", X_test.index.max())

Train size: (1148, 8)
Test size: (287, 8)
Train date range: 2022-01-02 00:00:00 → 2025-02-22 00:00:00
Test date range: 2025-02-23 00:00:00 → 2025-12-06 00:00:00


In [33]:
model = LogisticRegression(max_iter=1000,class_weight='balanced')
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [34]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.49477351916376305
[[96 47]
 [98 46]]
              precision    recall  f1-score   support

           0       0.49      0.67      0.57       143
           1       0.49      0.32      0.39       144

    accuracy                           0.49       287
   macro avg       0.49      0.50      0.48       287
weighted avg       0.49      0.49      0.48       287



Model has poor accuracy; attempt to optimise using scaling

In [35]:
from sklearn.preprocessing import StandardScaler

In [36]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
model_scaled = LogisticRegression(max_iter=1000)
model_scaled.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [38]:
y_pred_scaled = model_scaled.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_scaled))
print(confusion_matrix(y_test, y_pred_scaled))
print(classification_report(y_test, y_pred_scaled))


Accuracy: 0.5087108013937283
[[98 45]
 [96 48]]
              precision    recall  f1-score   support

           0       0.51      0.69      0.58       143
           1       0.52      0.33      0.41       144

    accuracy                           0.51       287
   macro avg       0.51      0.51      0.49       287
weighted avg       0.51      0.51      0.49       287

